# Sleep EEG cleaning

## Introductory notes:
This notebook presents cleaning functionality:
* Resampling
* Bandpass and notch filtering
* Selecting bad channels
* Interpolating bad channels 
* Annotating bad data spans

Recommended readings:
1. [MNE: The Raw data structure](https://mne.tools/stable/auto_tutorials/raw/10_raw_overview.html)
2. [Learning eeg: artifacts](https://www.learningeeg.com/artifacts)
3. [MNE: Overview of artifact detection](https://mne.tools/stable/auto_tutorials/preprocessing/10_preprocessing_overview.html)
4. [MNE: Filtering and resampling data](https://mne.tools/stable/auto_tutorials/preprocessing/30_filtering_resampling.html) 
5. [MNE: Handling bad channels](https://mne.tools/stable/auto_tutorials/preprocessing/15_handling_bad_channels.html)
6. [MNE: Annotating continuous data](https://mne.tools/stable/auto_tutorials/raw/30_annotate_raw.html)

## Import data

### Import module
Add pipeline module to path and import its elements (just run this cell).

In [1]:
from sleepeeg.pipeline import CleaningPipe

### Initialize CleaningPipe object

In [2]:
pipe = CleaningPipe(
    # Can be any type of eeg files supported by the mne.read_raw() function.
    path_to_eeg=r"C:\Users\Gennadiy\Documents\data\HZ4\HZ4_SLEEP_20210629_132715.mff",
    # A directory you want the results to be saved in.
    output_dir=r"C:\Users\Gennadiy\Documents\data\HZ4\processing"  
    )

Reading EGI MFF Header from C:\Users\Gennadiy\Documents\data\HZ4\HZ4_SLEEP_20210629_132715.mff...
    Reading events ...
    Assembling measurement info ...


## Resample

Resampling can be a long process (1+ hour), be patient.

In [3]:
pipe.mne_raw.load_data()

Reading 0 ... 8919090  =      0.000 ...  8919.090 secs...


<RawMff | signal1.bin, 264 x 8919091 (8919.1 s), ~17.54 GB, data loaded>

In [4]:
pipe.resample(
    # Whether to save the resampled eeg as .fif file. Will be saved in the {output_dir}/CleaningPipe/
    save=True,  
    mne_resample_args=dict(  # Arguments passed to the mne's raw.resample()
        sfreq=250,  # Desired new sampling frequency
        n_jobs='cuda'
    )
)

Now using CUDA device 0
Enabling CUDA with 10.97 GB available memory
Using CUDA for FFT resampling
Writing C:\Users\Gennadiy\Documents\data\HZ4\processing\CleaningPipe\resampled_250hz_raw.fif
    Writing channel names to FIF truncated to 15 characters with remapping
Overwriting existing file.
Writing C:\Users\Gennadiy\Documents\data\HZ4\processing\CleaningPipe\resampled_250hz_raw-1.fif
    Writing channel names to FIF truncated to 15 characters with remapping
Closing C:\Users\Gennadiy\Documents\data\HZ4\processing\CleaningPipe\resampled_250hz_raw-1.fif
Closing C:\Users\Gennadiy\Documents\data\HZ4\processing\CleaningPipe\resampled_250hz_raw.fif
[done]


## Filter

#### High pass

In [5]:
pipe.filter(
    mne_filter_args=dict(  # Arguments passed to the mne's raw.filter()
        l_freq=0.3,
        h_freq=None,
        picks=None,  # If None - filters all channels.
        n_jobs='cuda'
    )
)

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.3 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.30
- Lower transition bandwidth: 0.30 Hz (-6 dB cutoff frequency: 0.15 Hz)
- Filter length: 2751 samples (11.004 sec)

Using CUDA for FFT FIR filtering


#### Notch

In [6]:
# By default will remove 50 Hz and its harmonics.
pipe.notch(
    mne_notch_args=dict(  # Arguments passed to the mne's raw.notch_filter()
        picks='eeg',  # If None - filter all channels
        n_jobs='cuda'
    )
)

Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1651 samples (6.604 sec)

Using CUDA for FFT FIR filtering


## Select bad channels & annotate bad epochs

Create average reference projection. You can apply and remove the projection from inside the plot. Does not have an effect on raw signal itself.

In [7]:
pipe.mne_raw.set_eeg_reference(
    ref_channels='average',
    projection=True)

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


<RawMff | signal1.bin, 264 x 2229773 (8919.1 s), ~4.39 GB, data loaded>

#### Select bad channels

In [17]:
pipe.plot(
    save_bad_channels=True,  # Whether to save selected bad channels in a file
    save_annotations=False,  # Whether to save annotations in a file.
    overwrite=False,  # Whether to overwrite already saved bad_channels.txt or annotations.txt
    mne_plot_args=dict(  # Arguments passed to mne's raw.plot() function.
        scalings="auto",
        theme="dark",
        bad_color="r",
        use_opengl=False,
    )
)

Channels marked as bad:
['E229', 'E232', 'E181', 'E209', 'E217', 'E91']


If you want to continue with previously saved bad channels, use `pipe.read_bad_channels()`. The function will import the channels from *bad_channels.txt* file.

In [18]:
pipe.read_bad_channels(
    # Path to the txt file with bad channel name per row. 
    # If None set to '{output_dir}/CleaningPipe/bad_channels.txt'
    path=None
)

#### Interpolate bad channels

Interpolate bad channels using [spherical spline interpolation](https://mne.tools/stable/overview/implementation.html#bad-channel-repair-via-interpolation)

`reset_bads` - we saved bad channels in a txt file, so we can reset them in the metadata.

In [19]:
pipe.mne_raw.interpolate_bads(
    reset_bads=True  # Whether to set interpolated channels back as normal.
)

Interpolating bad channels
    Automatic origin fit: head of radius 96.5 mm
Computing interpolation matrix from 220 sensor positions
Interpolating 37 sensors


<RawMff | signal1.bin, 264 x 2229773 (8919.1 s), ~4.39 GB, data loaded>

#### Select bad epochs

In [22]:
pipe.plot(
    save_bad_channels=False,  # Whether to save selected bad channels in a file
    save_annotations=True,  # Whether to save annotations in a file.
    overwrite=False,  # Whether to overwrite already saved bad_channels.txt or annotations.txt
    mne_plot_args=dict(  # Arguments passed to mne's raw.plot() function.
        butterfly=True,
        scalings="auto",
        theme="dark",
        use_opengl=False,
    )
)

Channels marked as bad:
none


If you want to continue with previously saved annotations, use `pipe.read_annotations()`. The function will import the annotations from *annotations.txt* file.

In [23]:
pipe.read_annotations(
    # Path to txt file with mne-style annotations. 
    # If None set to '{output_dir}/annotations.txt'
    path=None
)

## Save cleaned and annotated signal to the file

In [20]:
pipe.save_raw('cleaned_raw.fif')

Writing C:\Users\Gennadiy\Documents\data\HZ4\processing\CleaningPipe\cleaned_raw.fif
    Writing channel names to FIF truncated to 15 characters with remapping
Overwriting existing file.
Writing C:\Users\Gennadiy\Documents\data\HZ4\processing\CleaningPipe\cleaned_raw-1.fif
    Writing channel names to FIF truncated to 15 characters with remapping
Closing C:\Users\Gennadiy\Documents\data\HZ4\processing\CleaningPipe\cleaned_raw-1.fif
Closing C:\Users\Gennadiy\Documents\data\HZ4\processing\CleaningPipe\cleaned_raw.fif
[done]
